In [1]:
#Imports

import datetime
import requests
import pandas as pd
import hopsworks
import datetime
from pathlib import Path
import json
import re
import os
import warnings
warnings.filterwarnings("ignore")

In [15]:
with open('./project_api_key.txt', 'r') as file:
    os.environ["HOPSWORKS_API_KEY"] = file.read().rstrip()
    print("API Key is set")
    print("First 5 characters of API Key: ", os.environ["HOPSWORKS_API_KEY"][:5])

API Key is set
First 5 characters of API Key:  pwWjy


In [16]:
#Connect to Hopsworks to store data in the feature store
project = hopsworks.login()

project = hopsworks.get_current_project()

2024-12-17 20:29:40,238 INFO: Closing external client and cleaning up certificates.
Connection closed.
2024-12-17 20:29:40,243 INFO: Initializing external client
2024-12-17 20:29:40,243 INFO: Base URL: https://c.app.hopsworks.ai:443
2024-12-17 20:29:41,488 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1168540


In [17]:
#Test if connection is working
print("Project name: ", project.name)
print("Project id: ", project.id)

Project name:  palhagen
Project id:  1168540


In [18]:
#Try downloadin the data from yfinance for omx30 
#This is some of the data that we will use to create features

import yfinance as yf
import pandas as pd
import datetime

# Download data from yfinance
omx30 = yf.Ticker("^OMX")
omx30_data = omx30.history(period="max")
omx30_data = omx30_data.reset_index()
omx30_data.to_csv("omx30_data.csv")


In [19]:
#Load into a pandas dataframe
df = pd.read_csv("omx30_data.csv")
#Clean dataframe from unnecessary columns such as Unnamed: 0, Dividends and Stock Splits
df = df.drop(columns=['Unnamed: 0', 'Dividends', 'Stock Splits'])


In [6]:
#Check dataframe
df.head()

,Date,Open,High,Low,Close,Volume
0,2008-11-20 00:00:00+01:00,594.119995,594.119995,572.469971,587.710022,190921000
1,2008-11-21 00:00:00+01:00,587.710022,594.799988,560.940002,567.609985,188222800
2,2008-11-24 00:00:00+01:00,567.609985,629.229980,567.609985,626.460022,197755700
3,2008-11-25 00:00:00+01:00,626.460022,644.549988,614.369995,634.020020,258695400
4,2008-11-26 00:00:00+01:00,634.020020,638.789978,617.489990,628.859985,192007800


In [20]:
fs = project.get_feature_store() 
#Try uploding the data to the feature store
 #THe data is formulated as: Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
Omx30_fg = fs.get_or_create_feature_group(
    name="omx30", 
    description="Omx30 data for the last 20 years",
    version=1,
    primary_key=['Date'], 
    event_time='Date'
)

#Upload the data to the feature store. 
#Update data if it already exists
Omx30_fg.insert(df)

Uploading Dataframe: 100.00% |██████████| Rows 4036/4036 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: omx30_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1168540/jobs/named/omx30_1_offline_fg_materialization/executions


(Job('omx30_1_offline_fg_materialization', 'SPARK'), None)

In [30]:
#test of fetching news data from the news api
from pygooglenews import GoogleNews
import json
import time 

gn = GoogleNews()
search = gn.search('Sweden finance', when='7d') 
news_data = search['entries']

news_data = pd.DataFrame(news_data)
news_data.to_csv("news_data.csv")

#we want to take only the headlines and the daata from the data
news_titles = news_data['title']

#Save titles to a file
news_titles.to_csv("news_titles.csv")

news_data.head()

,title,title_detail,links,link,id,guidislink,published,published_parsed,summary,summary_detail,source,sub_articles
0,Sweden's north frets over financial risks as g...,"{'type': 'text/plain', 'language': None, 'base...","[{'rel': 'alternate', 'type': 'text/html', 'hr...",https://news.google.com/rss/articles/CBMisAFBV...,CBMisAFBVV95cUxQUnp6Q3JVNllHbkV0VE9RcUhkN0E2cW...,False,"Wed, 18 Dec 2024 17:27:01 GMT","(2024, 12, 18, 17, 27, 1, 2, 353, 0)","<a href=""https://news.google.com/rss/articles/...","{'type': 'text/html', 'language': None, 'base'...","{'href': 'https://www.reuters.com', 'title': '...",[]
1,Klarna Tells Employees It Will Start Drug Test...,"{'type': 'text/plain', 'language': None, 'base...","[{'rel': 'alternate', 'type': 'text/html', 'hr...",https://news.google.com/rss/articles/CBMijwFBV...,CBMijwFBVV95cUxNemNOUkdYSS1jMGh2c2VpWHBRRTlxeE...,False,"Mon, 16 Dec 2024 17:51:00 GMT","(2024, 12, 16, 17, 51, 0, 0, 351, 0)","<a href=""https://news.google.com/rss/articles/...","{'type': 'text/html', 'language': None, 'base'...","{'href': 'https://www.businessinsider.com', 't...",[]
2,Sweden's finance ministry cuts economic growth...,"{'type': 'text/plain', 'language': None, 'base...","[{'rel': 'alternate', 'type': 'text/html', 'hr...",https://news.google.com/rss/articles/CBMiwgFBV...,CBMiwgFBVV95cUxNMHNCcTJZNDVhYkdQRmpiWjdqY3hsTm...,False,"Wed, 18 Dec 2024 11:49:45 GMT","(2024, 12, 18, 11, 49, 45, 2, 353, 0)","<a href=""https://news.google.com/rss/articles/...","{'type': 'text/html', 'language': None, 'base'...","{'href': 'https://www.investing.com', 'title':...",[]
3,Sweden open to power cable project if Germany ...,"{'type': 'text/plain', 'language': None, 'base...","[{'rel': 'alternate', 'type': 'text/html', 'hr...",https://news.google.com/rss/articles/CBMicEFVX...,CBMicEFVX3lxTE5qRF81VlNkckN4cVJhSU5LN3FmMS1GUT...,False,"Tue, 17 Dec 2024 05:01:28 GMT","(2024, 12, 17, 5, 1, 28, 1, 352, 0)","<a href=""https://news.google.com/rss/articles/...","{'type': 'text/html', 'language': None, 'base'...","{'href': 'https://www.ft.com', 'title': 'Finan...",[]
4,Swedish Government Cuts Growth View as Economi...,"{'type': 'text/plain', 'language': None, 'base...","[{'rel': 'alternate', 'type': 'text/html', 'hr...",https://news.google.com/rss/articles/CBMirwFBV...,CBMirwFBVV95cUxQRWVlNFB0MGdlSEJ2aGtuSlFLdVl0cT...,False,"Wed, 18 Dec 2024 11:10:05 GMT","(2024, 12, 18, 11, 10, 5, 2, 353, 0)","<a href=""https://news.google.com/rss/articles/...","{'type': 'text/html', 'language': None, 'base'...","{'href': 'https://www.bloomberg.com', 'title':...",[]
